In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [8]:
data = pd.read_csv('Dummies.csv')
data.head()

,Avant droite,Passager arrière (à droite),Passager arrière (à gauche),Passager arrière (au milieu),Passager avant (au milieu),Passager central (à gauche),Passager central (au mileu),Passager central (à droite),Passager,Piéton,...,3 voies de circulation,4 voies de circulation,5 voies de circulation,6 voies de circulation,7 voies de circulation,8 voies de circulation,9 voies de circulation,10 voies de circulation,11 voies de circulation,12 voies de circulation
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [38]:
targets = ['Tué', 'Indemne', 'Blessé léger', 'Blessé hospitalisé']
X = data.drop(targets, axis=1)
X = sm.add_constant(X)
X_opt = X.copy()
Y = data[targets].values.argmax(axis=1)

In [41]:
from statsmodels.discrete.discrete_model import MNLogit
significance_level = .01

highest_pvalue = 1
while X_opt.columns.tolist() and highest_pvalue > significance_level:
    result = MNLogit(Y, X).fit()
    highest_pvalue = result.pvalues.max()
    less_significant = result.pvalues.index[result.pvalues.values.argmax()]
    if highest_pvalue > significance_level:
        X_opt.drop(inplace=True, columns=less_significant)
        print(f'Eliminated {less_significant} (p-value: {highest_pvalue})')

C:\Users\Erwan\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2290: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\Erwan\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:2291: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]


Optimization terminated successfully.
         Current function value: nan
         Iterations 19


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
X_opt

Voir aussi `sklearn.feature_selection.SelectKBest`